In [11]:
from ncpol2sdpa import*
import numpy as np
import random
import pandas as pd
import smcp
import matplotlib as plt

In [2]:
data=pd.read_csv('FairOutput.csv',header=None)
Y=data.to_numpy()

In [3]:
# Set Parameters
level = 1
group = 2
trajectory = [2,3]
sum_traj = sum(trajectory)

In [28]:
for T in range(5,31):
    start=[0]+random.sample(range(0,3),trajectory[0]-1)+[0]+random.sample(range(0,2),trajectory[1]-1)
    end=random.sample(range(T-3,T),trajectory[0]-1)+[T]+random.sample(range(T-3,T),trajectory[1]-1)+[T]
    length=[end[i]-start[i] for i in range(sum_traj)]

# Decision Variables
    G = generate_operators("G", n_vars=1, hermitian=False, commutative=False)[0]
    Fdash = generate_operators("Fdash", n_vars=1, hermitian=False, commutative=False)[0]
    z = generate_operators("z", n_vars=1, hermitian=False, commutative=False)[0]
    m = generate_operators("m", n_vars=T+1, hermitian=False, commutative=False)
    q = generate_operators("q", n_vars=T, hermitian=False, commutative=False)
    p = generate_operators("p", n_vars=T, hermitian=False, commutative=False)
    f = generate_operators("f", n_vars=T, hermitian=False, commutative=False)

# Constraints
    ine1 = [f[i] - Fdash*m[i+1] - p[i] for i in range(T)]
    ine2 = [-f[i] + Fdash*m[i+1] + p[i] for i in range(T)]
    ine3 = [m[i+1] - G*m[i] - q[i] for i in range(T)]
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in range(T)]
    max1 = [z-1/trajectory[0]*sum(1/length[j]*sum((Y[t,j]-f[t])**2 for t in range(start[j],end[j])) for j in range(0,trajectory[0]))]
    max2 = [z-1/trajectory[1]*sum(1/length[j]*sum((Y[t,j]-f[t])**2 for t in range(start[j],end[j])) for j in range(trajectory[0],sum_traj))]
    ines_A = ine1+ine2+ine3+ine4+max1+max2

# Objective
    obj_A = z + 1*sum(p[i]**2 for i in range(T)) # 5 is optimal for level 1

# Solve the fair NCPO A
# fair A has the same size as fair B
    sdp_A = SdpRelaxation(variables = flatten([G,Fdash,z,f,p,m,q]), verbose = 2)
    sdp_A.get_relaxation(level, objective=obj_A, inequalities=ines_A)
    sdp_A.write_to_file(str(T)+'A.dat-s')


The problem has 24 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 1224
Generating moment matrix...
Reduced number of SDP variables: 1224224 (done: 100.08%, ETA 00:00:-0.0)
Processing 22/22 constraints...
The problem has 28 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 1652
Generating moment matrix...
Reduced number of SDP variables: 1652652 (done: 100.06%, ETA 00:00:-0.0)
Processing 26/26 constraints...
The problem has 32 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 2144
Generating moment matrix...
Reduced number of SDP variables: 2144144 (done: 100.05%, ETA 00:00:-0.0)
Processing 30/30 constraints...
The problem has 36 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 2700
Generating moment matrix...
Reduced number of SDP variables: 2700700 (done: 100.04%, ETA 00:00:-0.0)
Processi

'\n# write\noutputs=open(\'CompareOutput.txt\',\'w\')\nprint(nrmse,file=outputs)\nprint(nrmse_unfair,file=outputs)\noutputs.close()\n\n\n# write\na=10\noutputs=open(str(a)+\'.txt\',\'w\')\nprint("hello",file=outputs)\noutputs.close()\n'

In [30]:
Pm=[]
for T in range(5,31):
    P = smcp.SDP(str(T)+'A.dat-s')
    Pm.append(P.m)
#smcp.analysis.clique_hist(P)
#smcp.analysis.spy(P)

In [33]:
# write
outputs=open('mDim.csv','w')
print(Pm,file=outputs)
outputs.close()